In [10]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [11]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [12]:
!pip install sqlalchemy

In [13]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [14]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [15]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [16]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [17]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [18]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [19]:
# Find the most recent date in the data set.
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
recent_date

('2017-08-23',)

In [20]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
recent_date = recent_date[0]
# Use Pandas to calculate the summary statistics for the precipitation data
summary_statistics = df['Precipitation'].describe()
pd.DataFrame(summary_statistics)
# Calculate the date one year from the last date in data set.
recent_date = pd.to_datetime(recent_date)
recent_date = recent_date.to_pydatetime()

# Perform a query to retrieve the data and precipitation scores
one_year_ago = recent_date - dt.timedelta(days=366)
prcp = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > one_year_ago).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df = pd.DataFrame(prcp, columns=['Date', 'Precipitation'])

# Sort the dataframe by date
df = df.sort_values(by="Date")

# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x='Date', y='Precipitation', kind='line')
plt.xticks(rotation = 90)
plt.ylabel("Inches")

plt.show()

NameError: name 'df' is not defined

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
summary_statistics = df['Precipitation'].describe()
pd.DataFrame(summary_statistics

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
total_stations_query = session.query(Measurement.station).distinct().count()

total_stations_query

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.

# Query to count the number of measurements per station, sorted in descending order
most_active_stations = session.query(
    Measurement.station, 
    func.count(Measurement.station).label('count')
).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

# Print the results
for station, count in most_active_stations:
    print(f"Station: {station}, Count: {count}")

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

active_station = 'USC00519281'

temp_stats = session.query(func.min(Measurement.tobs).label('min_temp'),
                           func.max(Measurement.tobs).label('max_temp'),
                           func.avg(Measurement.tobs).label('avg_temp')
                          ).filter(Measurement.station == active_station).all()

min_temp, max_temp, avg_temp = temp_stats[0]
print(f"Lowest Temperature: {min_temp}")
print(f"Highest Temperature: {max_temp}")
print(f"Average Temperature: {avg_temp}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
#one_year_ago = datetime.now() - timedelta(days=365)

# Query the previous 12 months of TOBS data for the most active station
tobs_data = session.query(Measurement.tobs).filter(
    Measurement.station == active_station,
    Measurement.date >= one_year_ago
).all()

# Convert the result into a list of temperature observations
temperatures = [temp[0] for temp in tobs_data]

# Plotting the temperature data as a histogram with 12 bins
plt.figure(figsize=(10, 6))
plt.hist(temperatures, bins=12, color='blue', label = "Tobs")

# Adding labels and title
plt.xlabel("Temperature", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(loc='upper right')

# Show the plot
plt.show()

# Close Session

In [ ]:
# Close Session
session.close()